##Adversarial Machine Learning Library(Ad-lib)


## Introduction

### Overview
The Adversarial Machine Learning Library(Ad-lib) is a library that models the combats between spam-email attackers and robust spam-filters, using the adversarial machine learning methods. This library includes a data processing units, several attackings algorithms such as Restrained_attacks, Cost_sensitive_attacks, and Coordinate_greedy attackers, a simple learner wrap-ups and several robust learning algorithms.

### Dependencies
Ad-lib should be operated using Python 3.5 and above. The necessary dependencies are:
1. pyyaml
2. numpy
3. scipy
4. sklearn
5. cvxpy



##Data Processor
Raw email data is collected and processed by the EmailDataset class in data_reader.dataset.py, which uses Tfidvectorizer class from sklearn.feature_extraction(please refer to http://scikit-learn.org/stable/modules/generated/sklearn.feature_extraction.text.TfidfVectorizer.html for more details). The EmailDataset also enables battle/prediction loading and saving.

###Demo
Here we demonstrate: 
1. the loading of email data 
2. load the EmailDataset into instances(with a label, and feature vector that contains either binary or real values)
3. splitting the data into training instances and testing instances


In [ ]:
from data_reader.dataset import EmailDataset
from data_reader.operations import load_dataset

dataset = EmailDataset(path='./data_reader/data/raw/trec05p-1/test-400',binary= False,raw=True)
training_, testing_ = dataset.split({'train': 60, 'test': 40})
training_data = load_dataset(training_)
testing_data = load_dataset(testing_)

The EmailDataset() has several parameters: 
1. binary: a boolean varaiable that specifies whether our processed instances are binary or real value. 
2. raw: is true if we process raw email data. 
3. path: specifies the location of our files. 
4. features (scipy.sparse.csr_matrix, optional): dataset feature matrix 
5. labels (numpy.ndarray, optional): dataset labels corresponding to the feature matrix.
5. strip_accents_,ngram_range_, max_df_, min_df_, max_features_, num_instances: variables used by Tfidvectorizer.

##Attackers 
Many attackers assume that we have a basic knowledge of the learner. To succesfully initialize an attacker, we need to know the exact parameters used in the algortihms, as well as the some learners' features. This is sometimes achieved by calling set_adversarial_params.

###Attackers Demo
The following is a general demo of the Coordinate_Greedy attacker: 


In [ ]:
from sklearn import svm
from learners import SimpleLearner
import learners as learner
from data_reader.dataset import EmailDataset
from data_reader.operations import load_dataset
from adversaries.coordinate_greedy import CoordinateGreedy

#data processing unit
#the path is an index of 400 testing samples(raw email data).
dataset = EmailDataset(path='./data_reader/data/raw/trec05p-1/test-400',binary=True,raw=True)
training_, testing_ = dataset.split({'train': 60, 'test': 40})
training_data = load_dataset(training_)
testing_data = load_dataset(testing_)

#setting the default learner
#test simple learner svm
learning_model = svm.SVC(probability=True, kernel='linear')
learner1 = SimpleLearner(learning_model, training_data)
learner1.train()

#setting the coordinate_greedy attacker
attacker = CoordinateGreedy()
attacker.set_adversarial_params(learner=learner1, train_instances=training_data)
new_testing_data = attacker.attack(testing_data)

After the attack, the predictions of our learner model will change because we've disguised some spam emails(postivie instances) into benign
instances. We will introduce the accuracy test in the later part.



###Binary & Coordinate Greedy 

The binary and coordinate greedy algorithms are a general local search framework introduced by Bo Li, Yevgeniy Vorobeychik and Xinyun Chen in A General Retraining Framework for Scalable Adversarial Classification. The high-level idea is to randomly choose a feature, and greedily update the feature and increase the attacker's utility. The algorithm is proven to reach a global optimum as the number of iteration increases.

Please refer to https://arxiv.org/pdf/1604.02606.pdf for more details.

The general parameters of Binary_Greedy and Coordinate_Greedy algorithm includes:
1. learner: the learner algorithm(can be an approximation if the learner is not known)
2. max_change: maximum number of changes that can be made
3. lambda_val: weight in quodratic distances calculation
4. epsilon: the limit of difference between transform costs of ,xij+1, xij, and orginal x
5. step_size: weight for coordinate descent

Note: Binary Greedy algorithm only works for binary-value features, while Coordinate Greedy ony works in real-value settings.


###Cost Sensitive 

The cost sensitive attacker is an attacker that modifies malicious data with the consideration of minimizing the attacker cost. It is based on the Aderversarial Classification by Dalvi, Domingos, Mausam, Sanghai, and Verma. (https://homes.cs.washington.edu/~pedrod/papers/kdd04.pdf)

Given complete information about initial classifier C and an instance X, adversary finds a feature change strategy A(x) that maximizes its own utility Ua. By modeling the problem as a COP, which can be formulated as an integer linear program, the adversary finds the minimum cost camoflauge (MCC), or smallest cost of feature changes that covers the log odds gap between the classifier classifying the instance as positive and classifying it as a false negative. Only changes instances the classifier classified as positive and that can be changed without costing so much that it outweighs the utility that would be gained by C falsely classifying the instance as negative. 

The general parameters of Cost_Sensitive algorithm includes:
1. Ua: Utility accreued by Adversary when the classifier classifies as yc an instance of class y. 
       U(-,+) > 0, U(+,+)<0 ,and U(-,-)= U(+,-) = 0
2. Vi: Cost of measuring Xi
3. Uc: Utility of classifying yc an instance with true class y.
       Uc(+,-) <- and Uc(-,+) <0, Uc(+,+) >0, Uc(-,-) >0
4. Wi: Cost of changing the ith feature from xi to xi_prime
5. learner: the learner algorithm(should be a naive_bayes classifier)
6. scenario: can select three spam filtering scenarios: add words, add length, synonym

###Feature Deletion

The feature deletion algorithm is based on Nightmare at Test Time: Robust Learning by Feature Deletion by Amir Globerson
  and Sam Roweis.(http://people.csail.mit.edu/gamir/pubs/fdrop_camera_postfix.pdf)
  
It attempts to model a typical attacker that tries to delete the features with the least weights by sorting the features according to the weights and setting the features' value to zero to fool the learning algorithm. This algorithm only works for the binary-value instances.

general parameters includes:
1. learner: the learner algorithm. Note: this algorithm requires knowledge about feature weights from the learner.
2. num_deletion: the max number that will be deleted in the attack
3. all_malicious: if the flag is set, only features that are malicious will be deleted.

###Free Range Attack & Restrained attack

The free_range attacker and restrained attacker are generalized attacking algorithms proposed by the Adversarial Support Vector Machine Learning by Yan Zhou, Murat Kantarcioglu,Bhavani Thuraisingham and Bowei Xi.(http://www.utdallas.edu/~muratk/publications/kdd2012.pdf)

Both are algorithms that attempt to move the instances' features in a certain direction by some distance that is measured by how harsh the attack is. The algortihms differ in that the free_range attaker assumes that the adversary has the freedom to move anywhere in the feature space, while the restrained_attacker is more conservative. The algorithms can be used for both binary_value instances and real_value instances.

general parameters includes:
1. f_attack:float (between 0 and 1),determining the agressiveness of the attack.
2. xj_min:minimum xj that the feature can have. If not specified, it is calculated by going over all training data.(for free_range only)
3. xj_max:maximum xj that the feature can have. If not specified, it is calculated by going over all training data.(for free_range only)
4. binary: True means binary features.
5. learner: learner algorithm
6. type: specify how to find innocuous target
7. discount_factor:float(between 0 and 1),determing the data movement of the attack(for restrained only)


###SVM Restrained and SVM Free range learners

The SVM Restrained and SVM Free range learners are robust learners algorithms proposed by the Adversarial Support Vector Machine Learning by Yan Zhou, Murat Kantarcioglu,Bhavani Thuraisingham and Bowei Xi.(http://www.utdallas.edu/~muratk/publications/kdd2012.pdf)

The learers attempt to solve asymmetric dual problem: 'argmin (1/2)*⎜⎜w⎟⎟^2 + C*∑(xi0)`.
While the SVM Restraind learners consider only the aggressiveness of the attacker and deals with Restrained_attack, SVM Free Range can be used under Freerange_attack and provides satisfactory results. By solving the convex optimization, optimal weight and bias matrices of both learners are computed to be used in the linear classification of the instances changed by the adversary.

The parameters of SVM_Freerange are:
1. c_f (float): aggressiveness assumption c_f ∈ [0.0,1.0]
   Note: 0.0 means no attack and 1.0 is most aggressive. Default:0.5
2. xmin (float): smallest value that a feature can take. Default:0.0
3. xmax (float): largest value that a feature can take. Default:1.0

The parameters of SVM_Restrained are:
1. c_delta: aggressiveness assumption c_delta ∈ [0.0,1.0]. Default:0.5

###Good Word Attack

The Good Word Attack is based on Good Word Attacks on Statistical Spam Filters by Daniel Lowd and Christopher Meek.
(http://citeseerx.ist.psu.edu/viewdoc/download?doi=10.1.1.130.9846&rep=rep1&type=pdf)

This algorithm tries to measure the weight of each words in the email lists and attempts to create a list of n good words. It starts by randomly picking two messages, each from postivie instances and negative instances. Then it builds a merely negative message and a merely positive message that differ by one words. Then it iterates to find the first-n-words and best-n-words that could be added on to disguise the filters in the passive attack described in the paper. This algorithm only applies to binary-value features.

genearl parameters of the algorithms include:
1. n: number of words needed
2. attack_model_type: choose the best-n or first-n algorithm


##Robust Learners

The robust learners try to correctly identify the spam emails and benign emails even with the existence of an attacker. We have also included a wrap-up for simple learner in the learners. This allows us to see and compare the efficiency of robust learning algorithms. The simple learner's underlying classfication methods can also be switched by evoking different learners from sklearn.

###Robust Learners Demo

The following is a demonstration of a simple learner:

In [ ]:
#Simple learner
learning_model = svm.SVC(probability=True, kernel='linear')
learner1 = SimpleLearner(learning_model, training_data)
learner1.train()

The following is a demonstration of a robust learner of SVM_restrained:

In [ ]:
from learners.svm_restrained import SVMRestrained
learner2= SVMRestrained(training_instances=training_data)
learner2.train()
learner2.predict(testing_data)

###SVM Restrained and SVM Free range learners

The SVM Restrained and SVM Free range learners are robust learners algorithms proposed by the Adversarial Support Vector Machine Learning by Yan Zhou, Murat Kantarcioglu,Bhavani Thuraisingham and Bowei Xi.(http://www.utdallas.edu/~muratk/publications/kdd2012.pdf)

The learers attempt to solve asymmetric dual problem: 'argmin (1/2)*⎜⎜w⎟⎟^2 + C*∑(xi0)`.
While the SVM Restraind learners consider only the aggressiveness of the attacker and deals with Restrained_attack, SVM Free Range can be used under Freerange_attack and provides satisfactory results. By solving the convex optimization, optimal weight and bias matrices of both learners are computed to be used in the linear classification of the instances changed by the adversary.

The parameters of SVM_Freerange are:
1. c_f (float): aggressiveness assumption c_f ∈ [0.0,1.0]
   Note: 0.0 means no attack and 1.0 is most aggressive. Default:0.5
2. xmin (float): smallest value that a feature can take. Default:0.0
3. xmax (float): largest value that a feature can take. Default:1.0

The parameters of SVM_Restrained are:
1. c_delta: aggressiveness assumption c_delta ∈ [0.0,1.0]. Default:0.5

###Adversary Aware learner

The adversary aware learner is based on the Adversarial Classification By Nilesh Dalvi, Pedro Domingos, Mausam, Sumit Sanghai, Deepak Verma. 

This learner is set based on the assumption that the attacker is a naive bayes based attacker and uses it s optimal strategy to modify test 
instances. It considers the effect of the attacker into navie_bayes classifier.

The general parameters of Adversary_aware learner are:
1. attacker: need to be CostSensitive Attacker. Otherwise, we need to import utility of attacker or utility of learner.
2. Utility: the utility of attacker or learner(Optional)

###Learner Retraining

The retraining algorithm is a general learning algorithm introduced by Bo Li, Yevgeniy Vorobeychik and Xinyun Chen in A General Retraining Framework for Scalable Adversarial Classification. 

Given a model used to train in the initial stage and access to make calls to adversarial transformation methods, proceeds by
classifying the the given set of instances. Allows the adversary to iteratively transform the initial set of bad instances. While the
adversary is capable of changing a negative instance to a positive instance, retrains and notifies the adversary of the change.

After the improvement finishes, the underlying learner model has been updated, and can be used in the default prediction method.

The parameters of Learner_retranining are:
1.  base_model:the basic leanrer model 
2.  training_instances 
3.  attacker
4.  iteration times
